In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 37.5 MB/s eta 0:00:00


In [5]:
# Import necessary libraries
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
import pywt
import time

# Define a function to load the data from the directories
def load_data_from_directory(directory_path, label):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]  # Only .txt files

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Define a function for wavelet decomposition
    def wavelet_decomposition(signal, wavelet='db4', level=5):
        coeffs = pywt.wavedec(signal, wavelet, level=level)
        return np.concatenate(coeffs)

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)

        try:
            file_data = np.loadtxt(file_path)
            if file_data.ndim == 1:  # If 1D data, reshape to (n_samples, 1)
                file_data = file_data.reshape(-1, 1)

            # Preprocess the signal with wavelet decomposition
            preprocessed_data = wavelet_decomposition(file_data.flatten())
            data.append(preprocessed_data)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (Z vs. S only)
path_Z = '/content/drive/My Drive/PFA2_DataSet/DataSet/Healthy/Z'
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'

# Load data
data_Z, labels_Z = load_data_from_directory(path_Z, label=0)  # Healthy (Z)
data_S, labels_S = load_data_from_directory(path_S, label=1)  # Seizure (S)

# Combine data and labels for Z vs. S
if data_Z.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_Z, data_S), axis=0)
    labels_all = np.concatenate((labels_Z, labels_S), axis=0)

    # Flatten data arrays into 2D arrays and convert to float
    data_all = [d.flatten() for d in data_all]
    data_all = np.array(data_all, dtype=np.float32)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # One-hot encode the labels for binary classification
    y_train = to_categorical(y_train, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)

    # Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')  # 2 output classes (Healthy, Seizure)
    ])

    # Compile the model without an optimizer
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model and measure time
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)
    end_time = time.time()

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Compute evaluation metrics
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

    # Calculate sensitivity and specificity
    tp = conf_matrix[1, 1]  # True Positives
    tn = conf_matrix[0, 0]  # True Negatives
    fp = conf_matrix[0, 1]  # False Positives
    fn = conf_matrix[1, 0]  # False Negatives

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Print evaluation metrics
    print("\nFinal Evaluation Metrics:")
    print("Test Accuracy: ", accuracy)
    print("Test Recall: ", recall)
    print("Test Precision: ", precision)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)
    print("Confusion Matrix: \n", conf_matrix)
    print("Training Time (seconds): ", end_time - start_time)
else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")



    import os
from pathlib import Path

# Define the directory path
model_dir = "/content/drive/MyDrive/Models/Walvet_dec/ANN/Exp1"

# Create the directories if they don't exist
Path(model_dir).mkdir(parents=True, exist_ok=True)

# Save the model
model.save(os.path.join(model_dir, "No_Options.keras"))

print(f"Model saved successfully at {model_dir}/No_Options.keras ✅")



Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 295ms/step - accuracy: 0.4735 - loss: 162.8853 - val_accuracy: 0.5500 - val_loss: 30.1359
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 143ms/step - accuracy: 0.8632 - loss: 10.1694 - val_accuracy: 0.6667 - val_loss: 17.2537
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 249ms/step - accuracy: 0.9787 - loss: 0.3882 - val_accuracy: 0.6000 - val_loss: 27.6981
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 220ms/step - accuracy: 1.0000 - loss: 1.9798e-05 - val_accuracy: 0.6000 - val_loss: 27.6962
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step - accuracy: 1.0000 - loss: 7.5527e-06 - val_accuracy: 0.6000 - val_loss: 27.6940
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 194ms/step - accuracy: 1.0000 - loss: 2.1731e-05 - val_accuracy: 0.6000 - val_loss: 27.6921
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 1.0000 - loss: 1.7823e-05 - val_accuracy: 0.6000 - val_loss: 27.6900
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 1.0000 - loss: 1.4547e-05 - val

With Standerscaler

In [6]:
# Import necessary libraries
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import pywt
import time

# Define a function to load the data from the directories
def load_data_from_directory(directory_path, label):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]  # Only .txt files

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Define a function for wavelet decomposition
    def wavelet_decomposition(signal, wavelet='db4', level=5):
        coeffs = pywt.wavedec(signal, wavelet, level=level)
        return np.concatenate(coeffs)

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)

        try:
            file_data = np.loadtxt(file_path)
            if file_data.ndim == 1:  # If 1D data, reshape to (n_samples, 1)
                file_data = file_data.reshape(-1, 1)

            # Preprocess the signal with wavelet decomposition
            preprocessed_data = wavelet_decomposition(file_data.flatten())
            data.append(preprocessed_data)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (Z vs. S only)
path_Z = '/content/drive/My Drive/PFA2_DataSet/DataSet/Healthy/Z'
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'

# Load data
data_Z, labels_Z = load_data_from_directory(path_Z, label=0)  # Healthy (Z)
data_S, labels_S = load_data_from_directory(path_S, label=1)  # Seizure (S)

# Combine data and labels for Z vs. S
if data_Z.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_Z, data_S), axis=0)
    labels_all = np.concatenate((labels_Z, labels_S), axis=0)

    # Flatten data arrays into 2D arrays and convert to float
    data_all = [d.flatten() for d in data_all]
    data_all = np.array(data_all, dtype=np.float32)

    # Standardize the data
    scaler = StandardScaler()
    data_all = scaler.fit_transform(data_all)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # One-hot encode the labels for binary classification
    y_train = to_categorical(y_train, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)

    # Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')  # 2 output classes (Healthy, Seizure)
    ])

    # Compile the model without an optimizer
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model and measure time
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)
    end_time = time.time()

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Compute evaluation metrics
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

    # Calculate sensitivity and specificity
    tp = conf_matrix[1, 1]  # True Positives
    tn = conf_matrix[0, 0]  # True Negatives
    fp = conf_matrix[0, 1]  # False Positives
    fn = conf_matrix[1, 0]  # False Negatives

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Print evaluation metrics
    print("\nFinal Evaluation Metrics:")
    print("Test Accuracy: ", accuracy)
    print("Test Recall: ", recall)
    print("Test Precision: ", precision)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)
    print("Confusion Matrix: \n", conf_matrix)
    print("Training Time (seconds): ", end_time - start_time)
else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


    # Define the model path
model_path = "/content/drive/MyDrive/Models/Walvet_dec/ANN/Exp1/With_Stander.keras"

# Save the model
model.save(model_path)

print(f"Model saved successfully at {model_path} ✅")



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 126ms/step - accuracy: 0.5823 - loss: 0.6509 - val_accuracy: 0.9167 - val_loss: 0.2876
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9946 - loss: 0.0901 - val_accuracy: 0.9333 - val_loss: 0.1598
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.0100 - val_accuracy: 0.9333 - val_loss: 0.1749
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.0026 - val_accuracy: 0.9333 - val_loss: 0.1866
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 0.9167 - val_loss: 0.2093
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 1.0000 - loss: 9.6947e-04 - val_accuracy: 0.9167 - val_loss: 0.2219
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 6.4784e-04 - val_accuracy: 0.9000 - val_loss: 0.2381
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 1.0000 - loss: 4.5846e-04 - val_accuracy: 0.9000 - val_

With ONly optimiser

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

# Define a function to load the data from the directories
def load_data_from_directory(directory_path, label):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]  # Only .txt files

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)

        try:
            file_data = np.loadtxt(file_path)
            if file_data.ndim == 1:  # If 1D data, reshape to (n_samples, 1)
                file_data = file_data.reshape(-1, 1)
            data.append(file_data)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (Z vs. S only)
path_Z = '/content/drive/My Drive/PFA2_DataSet/DataSet/Healthy/Z'
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'

# Load data
data_Z, labels_Z = load_data_from_directory(path_Z, label=0)  # Healthy (Z)
data_S, labels_S = load_data_from_directory(path_S, label=1)  # Seizure (S)

# Combine data and labels for Z vs. S
if data_Z.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_Z, data_S), axis=0)
    labels_all = np.concatenate((labels_Z, labels_S), axis=0)

    # Flatten data arrays into 2D arrays and convert to float
    data_all = [d.flatten() for d in data_all]
    data_all = np.array(data_all, dtype=np.float32)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # One-hot encode the labels for binary classification
    y_train = to_categorical(y_train, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)

    # Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')  # 2 output classes (Healthy, Seizure)
    ])

    # Compile the model with Adam optimizer
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model and show training progress
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Compute evaluation metrics
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

    # Print evaluation metrics
    print("\nFinal Evaluation Metrics:")
    print("Test Accuracy: ", accuracy)
    print("Test Recall: ", recall)
    print("Test Precision: ", precision)
    print("Confusion Matrix: \n", conf_matrix)
else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


    # Define the model path
model_path = "/content/drive/MyDrive/Models/Walvet_dec/ANN/Exp1/With_Optimiser.keras"

# Save the model
model.save(model_path)

print(f"Model saved successfully at {model_path} ✅")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - accuracy: 0.5887 - loss: 106.3310 - val_accuracy: 0.4833 - val_loss: 107.2328
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.5177 - loss: 74.5730 - val_accuracy: 0.5833 - val_loss: 32.7494
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9484 - loss: 5.7428 - val_accuracy: 0.7667 - val_loss: 23.9250
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9491 - loss: 11.4951 - val_accuracy: 0.6833 - val_loss: 16.9294
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.9647 - loss: 0.6862 - val_accuracy: 0.5833 - val_loss: 38.3881
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9501 - loss: 0.9101 - val_accuracy: 0.6167 - val_loss: 45.6526
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.9868 - loss: 0.2615 - val_accuracy: 0.6167 - val_loss: 43.2166
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 1.5906e-05 - val_accuracy: 0.6167 - val_loss: 4

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step

Final Evaluation Metrics:
Test Accuracy:  0.6666666666666666
Test Recall:  0.675194660734149
Test Precision:  0.7333333333333334
Confusion Matrix: 
 [[13 18]
 [ 2 27]]
Model saved successfully at /content/drive/MyDrive/Models/Walvet_dec/ANN/Exp1/With_Optimiser.keras ✅


In [ ]:
# Import necessary libraries
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
import pywt
import time

# Define a function to load the data from the directories
def load_data_from_directory(directory_path, label):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]  # Only .txt files

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Define a function for wavelet decomposition
    def wavelet_decomposition(signal, wavelet='db4', level=5):
        coeffs = pywt.wavedec(signal, wavelet, level=level)
        return np.concatenate(coeffs)

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)

        try:
            file_data = np.loadtxt(file_path)
            if file_data.ndim == 1:  # If 1D data, reshape to (n_samples, 1)
                file_data = file_data.reshape(-1, 1)

            # Preprocess the signal with wavelet decomposition
            preprocessed_data = wavelet_decomposition(file_data.flatten())
            data.append(preprocessed_data)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (Z vs. S only)
path_Z = '/content/drive/My Drive/PFA2_DataSet/DataSet/Healthy/Z'
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'

# Load data
data_Z, labels_Z = load_data_from_directory(path_Z, label=0)  # Healthy (Z)
data_S, labels_S = load_data_from_directory(path_S, label=1)  # Seizure (S)

# Combine data and labels for Z vs. S
if data_Z.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_Z, data_S), axis=0)
    labels_all = np.concatenate((labels_Z, labels_S), axis=0)

    # Flatten data arrays into 2D arrays and convert to float
    data_all = [d.flatten() for d in data_all]
    data_all = np.array(data_all, dtype=np.float32)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # One-hot encode the labels for binary classification
    y_train = to_categorical(y_train, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)

    # Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')  # 2 output classes (Healthy, Seizure)
    ])

    # Compile the model with an optimizer
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model and measure time
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)
    end_time = time.time()

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Compute evaluation metrics
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

    # Calculate sensitivity and specificity
    tp = conf_matrix[1, 1]  # True Positives
    tn = conf_matrix[0, 0]  # True Negatives
    fp = conf_matrix[0, 1]  # False Positives
    fn = conf_matrix[1, 0]  # False Negatives

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Print evaluation metrics
    print("\nFinal Evaluation Metrics:")
    print("Test Accuracy: ", accuracy)
    print("Test Recall: ", recall)
    print("Test Precision: ", precision)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)
    print("Confusion Matrix: \n", conf_matrix)
    print("Training Time (seconds): ", end_time - start_time)
else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.3412 - loss: 112.3621 - val_accuracy: 0.5000 - val_loss: 95.3905
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7257 - loss: 20.3449 - val_accuracy: 0.5667 - val_loss: 50.0863
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.9486 - loss: 5.2337 - val_accuracy: 0.5667 - val_loss: 34.9031
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9896 - loss: 0.0108 - val_accuracy: 0.5000 - val_loss: 42.6733
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9458 - loss: 0.6835 - val_accuracy: 0.5000 - val_loss: 44.7693
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 4.0919e-04 - val_accuracy: 0.5500 - val_loss: 41.8386
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 1.0000 - loss: 1.4053e-07 - val_accuracy: 0.5500 - val_loss: 41.0933
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 4.6866e-08 - val_accuracy: 0.5333 - val_lo

Optimiser and Standarisation

In [ ]:
# Import necessary libraries
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import pywt
import time

# Define a function to load the data from the directories
def load_data_from_directory(directory_path, label):
    data = []
    labels = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} not found.")
        return np.array([]), np.array([])

    # List all files in the directory
    files = [f for f in os.listdir(directory_path) if f.lower().endswith('.txt')]  # Only .txt files

    # If no files found, return empty arrays
    if len(files) == 0:
        print(f"No .txt files found in {directory_path}.")
        return np.array([]), np.array([])

    # Define a function for wavelet decomposition
    def wavelet_decomposition(signal, wavelet='db4', level=5):
        coeffs = pywt.wavedec(signal, wavelet, level=level)
        return np.concatenate(coeffs)

    # Load data from each file
    for filename in files:
        file_path = os.path.join(directory_path, filename)

        try:
            file_data = np.loadtxt(file_path)
            if file_data.ndim == 1:  # If 1D data, reshape to (n_samples, 1)
                file_data = file_data.reshape(-1, 1)

            # Preprocess the signal with wavelet decomposition
            preprocessed_data = wavelet_decomposition(file_data.flatten())
            data.append(preprocessed_data)
            labels.append(label)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # Convert to numpy arrays
    data = np.array(data, dtype=object)
    labels = np.array(labels)

    return data, labels

# Paths to your datasets (Z vs. S only)
path_Z = '/content/drive/My Drive/PFA2_DataSet/DataSet/Healthy/Z'
path_S = '/content/drive/My Drive/PFA2_DataSet/DataSet/Seizeur/S'

# Load data
data_Z, labels_Z = load_data_from_directory(path_Z, label=0)  # Healthy (Z)
data_S, labels_S = load_data_from_directory(path_S, label=1)  # Seizure (S)

# Combine data and labels for Z vs. S
if data_Z.size > 0 and data_S.size > 0:
    data_all = np.concatenate((data_Z, data_S), axis=0)
    labels_all = np.concatenate((labels_Z, labels_S), axis=0)

    # Flatten data arrays into 2D arrays and convert to float
    data_all = [d.flatten() for d in data_all]
    data_all = np.array(data_all, dtype=np.float32)

    # Normalize data using StandardScaler
    scaler = StandardScaler()
    data_all = scaler.fit_transform(data_all)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.3, random_state=42)

    # One-hot encode the labels for binary classification
    y_train = to_categorical(y_train, num_classes=2)
    y_test = to_categorical(y_test, num_classes=2)

    # Define the ANN model
    model = Sequential([
        Dense(512, input_dim=X_train.shape[1], activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')  # 2 output classes (Healthy, Seizure)
    ])

    # Compile the model with an optimizer
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model and measure time
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)
    end_time = time.time()

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Compute evaluation metrics
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

    # Calculate sensitivity and specificity
    tp = conf_matrix[1, 1]  # True Positives
    tn = conf_matrix[0, 0]  # True Negatives
    fp = conf_matrix[0, 1]  # False Positives
    fn = conf_matrix[1, 0]  # False Negatives

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Print evaluation metrics
    print("\nFinal Evaluation Metrics:")
    print("Test Accuracy: ", accuracy)
    print("Test Recall: ", recall)
    print("Test Precision: ", precision)
    print("Sensitivity: ", sensitivity)
    print("Specificity: ", specificity)
    print("Confusion Matrix: \n", conf_matrix)
    print("Training Time (seconds): ", end_time - start_time)
else:
    print("Error: Data loading failed. Ensure the files are present in the directories.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 201ms/step - accuracy: 0.7322 - loss: 0.4693 - val_accuracy: 0.8833 - val_loss: 0.2970
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 1.0000 - loss: 0.0705 - val_accuracy: 0.8833 - val_loss: 0.2879
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 1.0000 - loss: 0.0055 - val_accuracy: 0.8333 - val_loss: 0.6784
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 1.0000 - loss: 3.8817e-04 - val_accuracy: 0.8333 - val_loss: 1.0126
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 7.3690e-05 - val_accuracy: 0.8333 - val_loss: 1.2346
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 1.0000 - loss: 2.0532e-05 - val_accuracy: 0.8333 - val_loss: 1.3761
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 1.0000 - loss: 1.3527e-05 - val_accuracy: 0.8333 - val_loss: 1.4632
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 6.5393e-06 - val_accuracy: 0.833